In [3]:
from module import models
from module import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
# Load dataset

df = datasets.amazon()
df

,name,main_category
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances
...,...,...
1103165,Adidas Regular Fit Men's Track Tops,sports & fitness
1103166,Redwolf Noice Toit Smort - Hoodie (Black),sports & fitness
1103167,Redwolf Schrute Farms B&B - Hoodie (Navy Blue),sports & fitness
1103168,Puma Men Shorts,sports & fitness


In [5]:
# Removing the numbers
df["name"] = df["name"].str.replace(r'\d+', '', regex=True)

In [6]:
# Removing the punctuations
df["name"] = df["name"].str.replace(r'[^\w\s]', '', regex=True)

In [7]:
# Stopword removal
def remove_stopwords(text):
    """Remove stopwords from text"""
    if pd.isna(text):
        return text
    
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Tokenize the text
    word_tokens = word_tokenize(text.lower())
    
    # Filter out stopwords
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

# Apply stopword removal
df["name"] = df["name"].apply(remove_stopwords)

In [8]:
df

,name,main_category
0,lloyd ton star inverter split ac convertible c...,appliances
1,lg ton star ai dual inverter split ac copper s...,appliances
2,lg ton star ai dual inverter split ac copper s...,appliances
3,lg ton star ai dual inverter split ac copper s...,appliances
4,carrier ton star inverter split ac copperester...,appliances
...,...,...
1103165,adidas regular fit mens track tops,sports & fitness
1103166,redwolf noice toit smort hoodie black,sports & fitness
1103167,redwolf schrute farms bb hoodie navy blue,sports & fitness
1103168,puma men shorts,sports & fitness


In [9]:
# BoW
cv = CountVectorizer()
X = cv.fit_transform(df["name"])
# Label Encoding
le = LabelEncoder()
y = le.fit_transform(df["main_category"])
# Model
lr = models.LogisticRegression(max_iter=1000)
lr = models.train_and_eval(X, y, classes=le.classes_, model=lr)

                         precision    recall  f1-score   support

            accessories       0.90      0.96      0.93     46242
             appliances       0.97      0.99      0.98     13209
         bags & luggage       0.79      0.62      0.70      4248
        beauty & health       0.93      0.93      0.93      4023
        car & motorbike       0.95      0.92      0.93      2818
grocery & gourmet foods       0.93      0.97      0.95      1337
         home & kitchen       0.91      0.89      0.90      5774
    industrial supplies       0.89      0.83      0.86      1624
          kids' fashion       0.87      0.79      0.83      5367
         men's clothing       0.96      0.99      0.97     30696
            men's shoes       0.89      0.96      0.92     23077
                  music       0.97      0.89      0.93       400
           pet supplies       0.99      0.97      0.98       646
       sports & fitness       0.82      0.71      0.76      4969
                 stores 

In [10]:
models.save_model(lr, "lr_full.pkl")
models.save_model(cv, "vectorizer.pkl")
models.save_model(le, "labelencoder.pkl")